# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import math
import torch
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
df.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 18 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   item_id               1503424 non-null  object 
 1   user_id               1503424 non-null  object 
 2   region                1503424 non-null  object 
 3   city                  1503424 non-null  object 
 4   parent_category_name  1503424 non-null  object 
 5   category_name         1503424 non-null  object 
 6   param_1               1441848 non-null  object 
 7   param_2               848882 non-null   object 
 8   param_3               640859 non-null   object 
 9   title                 1503424 non-null  object 
 10  description           1387148 non-null  object 
 11  price                 1418062 non-null  float64
 12  item_seq_number       1503424 non-null  int64  
 13  activation_date       1503424 non-null  object 
 14  user_type             1503424 non-

Выделим числовые поля из набора данных и выделим целевую переменную

In [4]:
dftrain = df[['price','item_seq_number','image_top_1']]
label = df['deal_probability']

Определим класс для описания датасета

In [6]:
class MyDate(torch.utils.data.Dataset):
   
    def __init__(self, df, target):
        self.df = torch.Tensor(np.array(df))
        self.label = torch.Tensor(np.array(target))

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        row = self.df[idx]
        target = self.label[idx]
        
        return row, target

In [7]:
dataset = MyDate(dftrain,label)

In [8]:
dataset[0]

(tensor([ 400.,    2., 1008.]), tensor(0.1279))

Разобьем на батчи

In [9]:
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=6,
                          shuffle=False)

In [10]:
for batch in (train_loader):
    print(batch[0])
    print(batch[1])
    break

tensor([[4.0000e+02, 2.0000e+00, 1.0080e+03],
        [3.0000e+03, 1.9000e+01, 6.9200e+02],
        [4.0000e+03, 9.0000e+00, 3.0320e+03],
        [2.2000e+03, 2.8600e+02, 7.9600e+02],
        [4.0000e+04, 3.0000e+00, 2.2640e+03],
        [1.3000e+03, 9.0000e+00, 7.9600e+02]])
tensor([0.1279, 0.0000, 0.4318, 0.8032, 0.2080, 0.8032])


Сверим с исходными данными

In [11]:
dftrain.head(6)

,price,item_seq_number,image_top_1
0,400.0,2,1008.0
1,3000.0,19,692.0
2,4000.0,9,3032.0
3,2200.0,286,796.0
4,40000.0,3,2264.0
5,1300.0,9,796.0


In [12]:
label[:6]

0    0.12789
1    0.00000
2    0.43177
3    0.80323
4    0.20797
5    0.80323
Name: deal_probability, dtype: float64

Включим перемешивание и увеличим batch_size

In [13]:
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=1024,
                          shuffle=True)

In [14]:
for batch in (train_loader):
    print(batch[0].shape, batch[1].shape )
    break

torch.Size([1024, 3]) torch.Size([1024])
